# Ejercicio 2 - Venta de coches KARLS

## Creación Base de Datos

In [1]:
import sqlite3

In [2]:
# Conexión a base de datos

conn = sqlite3.connect('karls.sqlite')
cursor = conn.cursor()

In [3]:
# Creación tabla Lineas de producto

query_creacionTabla = '''
CREATE TABLE LINEAS
    (ID INT NOT NULL,
    CODIGO TEXT NOT NULL,
    DESCRIPCION TEXT,
    PRIMARY KEY (ID, CODIGO)
    )
'''
cursor.execute(query_creacionTabla)

In [4]:
# Creación tabla Productos

query_creacionTabla = '''
CREATE TABLE PRODUCTOS
    (ID INT NOT NULL,
    CODIGO TEXT NOT NULL,
    NOMBRE TEXT,
    CODIGO_LINEA TEXT,
    PRECIO_COMPRA FLOAT,
    PRIMARY KEY (ID, CODIGO)
    )
'''
cursor.execute(query_creacionTabla)

In [5]:
# Creación tabla Clientes

query_creacionTabla = '''
CREATE TABLE CLIENTES
    (ID INT PRIMARY KEY NOT NULL,
    NOMBRE TEXT,
    TELEFONO INT,
    DIRECCION TEXT,
    CIUDAD TEXT,
    ESTADO TEXT,
    CODIGO_POSTAL TEXT,
    PAIS TEXT
    )
'''
cursor.execute(query_creacionTabla)

In [6]:
# Creación tabla Ventas

query_creacionTabla = '''
CREATE TABLE VENTAS
    (ID INT PRIMARY KEY NOT NULL,
    ID_ORDEN INT,
    CODIGO_PRODUCTO TEXT,
    PRECIO FLOAT,
    CANTIDAD INT
    )
'''
cursor.execute(query_creacionTabla)

In [7]:
# Creación tabla Ordenes, he añadido un campo más: Precio Total. Este campo le da más
# sentido a la tabla ordenes

query_creacionTabla = '''
CREATE TABLE ORDENES
    (ID INT PRIMARY KEY NOT NULL,
    FECHA TEXT,
    ID_CLIENTE INT,
    PRECIO_TOTAL FLOAT
    )
'''
cursor.execute(query_creacionTabla)

## Tratamiento de datos - Ventas.csv

In [8]:
import pandas as pd

In [9]:
# Importamos la informacion a un DataFrame

df = pd.read_csv('ventas.csv')

In [10]:
# Comprobamos que los tipos de las columnas sean correctos

df.dtypes

no_orden             int64
codigo_producto     object
cantidad_orden       int64
precio_cada        float64
nombre_producto     object
linea_productos     object
descr               object
precio_compra      float64
fecha_orden         object
no_cliente           int64
nombre_cliente      object
tlf                 object
direccion           object
ciudad              object
estado              object
codigo_postal       object
pais                object
dtype: object

## Almacenamos la información en la BBDD

In [11]:
# Primero, tenemos que introducir el campo precio_total en el dataframe

df['precio_total'] = df['precio_cada'] * df['cantidad_orden']

In [12]:
# Posteriormente, creamos un dataframe por cada tabla con los campos que necesitamos

df_lineas = pd.DataFrame({'CODIGO':df['linea_productos'], 'DESCRIPCION':df['descr']})

df_productos = pd.DataFrame({'CODIGO':df['codigo_producto'], 'NOMBRE':df['nombre_producto'],
                            'CODIGO_LINEA':df['linea_productos'], 'PRECIO_COMPRA':df['precio_compra']})

df_clientes = pd.DataFrame({'ID':df['no_cliente'], 'NOMBRE':df['nombre_cliente'],
                            'TELEFONO':df['tlf'], 'DIRECCION':df['direccion'],
                            'CIUDAD':df['ciudad'], 'ESTADO':df['estado'], 
                            'CODIGO_POSTAL':df['codigo_postal'],'PAIS':df['pais']})

df_ventas = pd.DataFrame({'ID':list(df.index.values + 1), 'ID_ORDEN':df['no_orden'],
                          'CODIGO_PRODUCTO':df['codigo_producto'],  'PRECIO':df['precio_cada'],
                          'CANTIDAD':df['cantidad_orden']})

df_ordenes = pd.DataFrame({'ID':df['no_orden'], 'FECHA':df['fecha_orden'],
                            'ID_CLIENTE':df['no_cliente']})

In [13]:
# Como va a haber duplicados ya que hemos desagregado los datos
# necesitamos borrar estos antes de almacenarlos en la base de datos

df_lineas = df_lineas.drop_duplicates()
df_productos = df_productos.drop_duplicates()
df_clientes = df_clientes.drop_duplicates()
df_ordenes = df_ordenes.drop_duplicates()

In [14]:
# Añadimos la columna precio total a ordenes

df_ordenes['PRECIO_TOTAL'] = list(df.groupby('no_orden')['precio_total'].sum())

In [15]:
# Añadimos los IDs a los dataframe que no los tienen

df_lineas.insert(0, 'ID', list(df_lineas.index.values + 1))
df_productos.insert(0, 'ID', list(df_productos.index.values + 1))

In [16]:
# Almacenamos en la base de datos la información correspondiente

df_lineas.to_sql('LINEAS', conn, index=False, if_exists='append')
df_productos.to_sql('PRODUCTOS', conn, index=False, if_exists='append')
df_clientes.to_sql('CLIENTES', conn, index=False, if_exists='append')
df_ventas.to_sql('VENTAS', conn, index=False, if_exists='append')
df_ordenes.to_sql('ORDENES', conn, index=False, if_exists='append')

326

## Acceso a la base de datos World

In [17]:
#Importamos la librería y nos conectamos con las credenciales correspondientes

import pymysql

database_host = 'relational.fit.cvut.cz'
username = 'guest'
password = 'relational'
database_name = 'world'

db = pymysql.connect(host=database_host,
                     user=username,
                     password=password,
                     database=database_name)

In [18]:
# Quitamos los warnings

import warnings
warnings.filterwarnings("ignore")

In [19]:
# Comprobamos que nos hemos conectado correctamente

pd.read_sql('SHOW TABLES', db)

,Tables_in_world
0,City
1,Country
2,CountryLanguage


In [20]:
# Definimos y ejecutamos una consulta para obtener el idioma por ciudad y pais

query = '''
SELECT Country.Name, City.Name, Language, Percentage
FROM Country
JOIN City
ON Country.Code = City.CountryCode
JOIN CountryLanguage
ON Country.Code = CountryLanguage.CountryCode
'''

df_languages = pd.read_sql(query, db)

In [21]:
df_languages

,Name,Name,Language,Percentage
0,Afghanistan,Kabul,Balochi,0.9
1,Afghanistan,Kabul,Dari,32.1
2,Afghanistan,Kabul,Pashto,52.4
3,Afghanistan,Kabul,Turkmenian,1.9
4,Afghanistan,Kabul,Uzbek,8.8
...,...,...,...,...
30665,Palestine,Jabaliya,Hebrew,4.1
30666,Palestine,Nablus,Arabic,95.9
30667,Palestine,Nablus,Hebrew,4.1
30668,Palestine,Rafah,Arabic,95.9


Observo que hay más de un idioma por ciudad, por lo que hay que establecer un criterio para definir el idioma elegido para cada cliente.

## Creando Tabla Idiomas-Cliente

El criterio de eleccion que voy a usar es según el "porcentaje" (tabla CountryLanguage) de uso del idioma en cada país.

In [22]:
#En la siguiente columna lo que hago es seleccionar el idioma de cada país que más gente habla. 

query = '''
SELECT DISTINCT Country.Name as Country, Language, Percentage
FROM Country
JOIN City
ON Country.Code = City.CountryCode
JOIN CountryLanguage cl1
ON Country.Code = cl1.CountryCode
WHERE Percentage = (SELECT MAX(Percentage)
                    FROM CountryLanguage cl2
                    WHERE cl1.CountryCode = cl2.CountryCode)
'''

df_languages = pd.read_sql(query, db)
df_languages

,Country,Language,Percentage
0,Aruba,Papiamento,76.7
1,Afghanistan,Pashto,52.4
2,Angola,Ovimbundu,37.2
3,Anguilla,English,0.0
4,Albania,Albaniana,97.9
...,...,...,...
235,Yemen,Arabic,99.6
236,Yugoslavia,Serbo-Croatian,75.2
237,South Africa,Zulu,22.7
238,Zambia,Bemba,29.7


Ya que no es una consulta trivial, voy a realizar varias consultas para comprobar si realmente tiene sentido el resultado de la anterior

In [23]:
# Primero voy a ver cuantos paises hay guardados en la base de datos

query = '''
SELECT DISTINCT Country.Name
FROM Country
'''

df_auxiliar = pd.read_sql(query, db)
df_auxiliar

,Name
0,Aruba
1,Afghanistan
2,Angola
3,Anguilla
4,Albania
...,...
234,Yemen
235,Yugoslavia
236,South Africa
237,Zambia


Se observa que en mi consulta hay 240 paises y que realmente en la base de datos hay 239 guardados, esto puede ser porque hay un país en el que se hablan dos idiomas con exactamente el mismo porcentaje. Voy a comprobar si realmente es así.

In [24]:
df_languages.groupby(["Country"]).count().loc[df_languages.groupby(["Country"]).count()['Language'] > 1]

,Language,Percentage
Country,,
Christmas Island,2,2
Cocos (Keeling) Islands,2,2
Cook Islands,2,2
East Timor,2,2
Niue,2,2
Svalbard and Jan Mayen,2,2
Tokelau,2,2
Wallis and Futuna,2,2


Se observa que hay varios paises que tienen dos idiomas con el mismo porcentaje. No me preocupa ya que son países que no son muy conocidos y es muy poco probable que alguno de los clientes resida en estos paises, además si comprobamos cualquiera de estos en el Dataframe de idiomas vemos que el porcentaje siempre es 0%, esto entiendo que debe ser porque no tienen población y por tanto, no nos afectará en nuestro caso

Sigo realizando consultas para comprobar que la consulta es correcta

In [25]:
# Voy a comprobar que realmente la consulta está cogiendo el máximo de los porcentajes dentro de cada idioma.
# Esto lo voy a hacer comprobando algunos de los países que hemos visto en el dataframe de lenguajes

query = '''
SELECT DISTINCT Country.Name as Country, Language, Percentage
FROM Country
JOIN City
ON Country.Code = City.CountryCode
JOIN CountryLanguage cl1
ON Country.Code = cl1.CountryCode
WHERE Country.Name = "Yugoslavia"
'''

df_auxiliar = pd.read_sql(query, db)
df_auxiliar

,Country,Language,Percentage
0,Yugoslavia,Albaniana,16.5
1,Yugoslavia,Hungarian,3.4
2,Yugoslavia,Macedonian,0.5
3,Yugoslavia,Romani,1.4
4,Yugoslavia,Serbo-Croatian,75.2
5,Yugoslavia,Slovak,0.7


Esta es correcta

In [26]:
query = '''
SELECT DISTINCT Country.Name as Country, Language, Percentage
FROM Country
JOIN City
ON Country.Code = City.CountryCode
JOIN CountryLanguage cl1
ON Country.Code = cl1.CountryCode
WHERE Country.Name = "Zimbabwe"
'''

df_auxiliar = pd.read_sql(query, db)
df_auxiliar

,Country,Language,Percentage
0,Zimbabwe,English,2.2
1,Zimbabwe,Ndebele,16.2
2,Zimbabwe,Nyanja,2.2
3,Zimbabwe,Shona,72.1


También correcta, una última comprobación:

In [27]:
query = '''
SELECT DISTINCT Country.Name as Country, Language, Percentage
FROM Country
JOIN City
ON Country.Code = City.CountryCode
JOIN CountryLanguage cl1
ON Country.Code = cl1.CountryCode
WHERE Country.Name = "Afghanistan"
'''


df_auxiliar = pd.read_sql(query, db)
df_auxiliar

,Country,Language,Percentage
0,Afghanistan,Balochi,0.9
1,Afghanistan,Dari,32.1
2,Afghanistan,Pashto,52.4
3,Afghanistan,Turkmenian,1.9
4,Afghanistan,Uzbek,8.8


También correcta, ahora voy a comprobar que los nombres de los paises sean los mismos en los dos origenes de datos.

In [28]:
paises_clientes = set(df_clientes['PAIS'])
paises_idiomas = df_languages[df_languages['Country'].isin(paises_clientes)]['Country']
paises_no_iguales = list(paises_clientes - set(paises_idiomas))
paises_no_iguales

['Norway  ', 'USA', 'UK']

Vemos que a Noruega, Reino Unido y Estados unidos no se les hace referencia de la misma forma, al ser solo tres, voy a cambiarlo directamente por ser más rápido y así hacer el merge de los clientes e idiomas directamente.

In [29]:
# Primero localizamos como se ha hecho referencia a estos paises en la BBDD world

# En el caso de Noruega es por el espacio despues del nombre

query = '''
SELECT DISTINCT Country.Name as Country, Language, Percentage
FROM Country
JOIN City
ON Country.Code = City.CountryCode
JOIN CountryLanguage cl1
ON Country.Code = cl1.CountryCode
WHERE Country.Name = "United States" OR Country.Name = "United Kingdom" OR Country.Name = "Norway"
'''


df_auxiliar = pd.read_sql(query, db)
df_auxiliar

,Country,Language,Percentage
0,United Kingdom,English,97.3
1,United Kingdom,Gaeli,0.1
2,United Kingdom,Kymri,0.9
3,Norway,Danish,0.4
4,Norway,English,0.5
5,Norway,Norwegian,96.6
6,Norway,Saame,0.0
7,Norway,Swedish,0.3
8,United States,Chinese,0.6
9,United States,English,86.2


In [30]:
# Modificamos los dataframe para poder hacer el merge posteriormente

df_languages.replace(['United States', 'United Kingdom'], ['USA', 'UK'], inplace = True)
df_clientes.replace(['Norway  '], ['Norway'], inplace = True)

In [31]:
# Realizo el merge para atribuirle a cada cliente su idioma a través del país de nacimiento

df_idiomas_clientes = df_clientes.merge(df_languages, how="left", left_on="PAIS", right_on="Country")

In [32]:
# Elimino las columnas que no me interesan 

df_idiomas_clientes.drop(columns=['NOMBRE', 'TELEFONO', 'DIRECCION', 
                                  'CIUDAD', 'ESTADO', 'CODIGO_POSTAL', 'PAIS', 'Country', 'Percentage'], inplace=True)

In [33]:
# Cambio el nombre de las columnas que voy a almacenar en la base de datos

df_idiomas_clientes.rename(columns={"ID":"ID_CLIENTE", "Language":"LANGUAGE"}, inplace=True)

In [34]:
# Creo la tabla en la BBDD y almaceno los datos

df_idiomas_clientes.to_sql('IDIOMAS_CLIENTE', conn, index=False, if_exists='replace')

98

In [35]:
# Para terminar este bloque compruebo que se ha guardado correctamente

query = '''
    SELECT *
    FROM IDIOMAS_CLIENTE
'''
cursor.execute(query)
cursor.fetchall()

[(363, 'English'),
 (128, 'German'),
 (181, 'English'),
 (121, 'Norwegian'),
 (141, 'Spanish'),
 (145, 'Danish'),
 (278, 'Italian'),
 (131, 'English'),
 (385, 'Pilipino'),
 (486, 'English'),
 (187, 'English'),
 (129, 'English'),
 (144, 'Swedish'),
 (124, 'English'),
 (172, 'French'),
 (424, 'English'),
 (381, 'Dutch'),
 (148, 'Chinese'),
 (216, 'Spanish'),
 (382, 'German'),
 (114, 'English'),
 (353, 'French'),
 (350, 'French'),
 (103, 'French'),
 (112, 'English'),
 (458, 'Spanish'),
 (151, 'English'),
 (324, 'English'),
 (198, 'English'),
 (447, 'English'),
 (323, 'English'),
 (250, 'French'),
 (242, 'French'),
 (496, 'English'),
 (282, 'English'),
 (161, 'English'),
 (334, 'Finnish'),
 (320, 'English'),
 (205, 'English'),
 (379, 'English'),
 (276, 'English'),
 (487, 'English'),
 (311, 'Finnish'),
 (333, 'English'),
 (219, 'English'),
 (186, 'Finnish'),
 (473, 'Italian'),
 (321, 'English'),
 (347, 'English'),
 (227, 'Danish'),
 (452, 'German'),
 (462, 'English'),
 (448, 'Swedish'),
 (2

## Opcional - Producto con más beneficios

In [36]:
query = '''
    SELECT CODIGO, NOMBRE, SUM(BENEFICIO_PRODUCTO * VECES_VENDIDO) AS BENEFICIO_TOTAL
    FROM (SELECT CODIGO, NOMBRE, PRECIO - PRECIO_COMPRA AS BENEFICIO_PRODUCTO, SUM(CANTIDAD) AS VECES_VENDIDO
        FROM PRODUCTOS
        JOIN VENTAS
        ON CODIGO = CODIGO_PRODUCTO
        GROUP BY CODIGO, NOMBRE, BENEFICIO_PRODUCTO) TABLA1
    GROUP BY CODIGO, NOMBRE
    ORDER BY BENEFICIO_TOTAL DESC
    LIMIT 1;   
'''
cursor.execute(query)
cursor.fetchall()

[('S18_3232', '1992 Ferrari 360 Spider red', 135996.78)]